In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from scipy.stats import chi2_contingency
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
#df= pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240326/full_step2.csv')
#df = pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240424/full_step2.csv')
#df = pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step2.csv')

#df = pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20250312/full_step2.csv')
df = pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step2.csv')


#df = pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/test_30007565.csv')
#df = df[:20]

In [3]:
#print(df['ICU_Class_2'].sum())

#print(df)


In [4]:
for col in df.columns:
    assert df[col].isnull().values.any() == False,f'{col}欄位有缺失值'

In [5]:
"""
輸入label
回傳正、負樣本索引
"""
def search_positive_negative_sample(y_list):
    #正負樣本
    positive_index = [i for i, arr in enumerate(y_list) if arr[0] == 1]
    negative_index = [i for i, arr in enumerate(y_list) if arr[0] == 0]
    
    
    # positive_count = len(positive_index)
    # negative_count = len(negative_index)
    
    # print(f"正樣本數量: {positive_count}")
    # print(f"負樣本數量: {negative_count}")   
    #print(f"正樣{head(10).positive_idex}")

    
    #有正樣本情況下，只保留前面樣本
    min_positive_index = min(positive_index) if positive_index else float('inf')
    negative_index = [i for i in negative_index if i < min_positive_index]
    return positive_index, negative_index


# for i in range(0, len(positive_index)):
#         i=i+1
# print("Value: ", i)
"""
移除部分樣本
>> 樣本數<=2  ->  全保留
>> 反之，正樣本全保留，負樣本只保留部分
"""
def remove_sample(x_list, y_list, task_name = 'Null'):
    assert len(x_list) == len(y_list)      
    if len(x_list) <=2:
        return x_list,y_list
    else:
        positive_index, negative_index = search_positive_negative_sample(y_list)
        if task_name == 'DNR' or 'dialysis':
            select_index = positive_index + random.sample(negative_index, k=min(1, len(negative_index)))          
        else:
            select_index = positive_index + random.sample(negative_index, k=min(2, len(negative_index)))
        random.shuffle(select_index)                             
                                                      
        return [x_list[i] for i in select_index], [y_list[i] for i in select_index] 

    

In [6]:
"""
3天內發生DNR都視為DNR
"""
def correction_DNR(df_P):
    df_P['max_DNR_old'] = df_P['DNR_old'].rolling(window=3, min_periods=1).max()
    df_P['DNR'] = np.where(df_P['max_DNR_old'] == 1, 1, 0)
    df_P.drop(columns='DNR_old', inplace=True)
    df_P.drop(columns='max_DNR_old', inplace=True)
    return df_P

"""
>>將第一天時序特徵擴充為每日靜態特徵

extend_col_num: 前幾個col要擴充成新的靜態特徵
"""
def add_fisrt_day_columns(df_P, extend_col_num = 2):
    # 創建新的欄位名稱列表
    day_columns = [f"first_day_{df_P.columns[i]}" for i in range(extend_col_num)] 

    # 提取前兩個欄位的資料作為新的欄位值
    df_first_day = pd.DataFrame([df_P.iloc[0, :extend_col_num].values], columns=day_columns) 
    df_first_day = pd.concat([df_first_day] * len(df_P), ignore_index=True)
    # 將新的欄位新增到每一行後面
    df_P = pd.concat([df_P, df_first_day], axis=1).reset_index(drop=True)
    return df_P

In [7]:
df_columns = pd.DataFrame()

print(df_columns)

Empty DataFrame
Columns: []
Index: []


In [8]:
"""
輸入患者資料、時窗大小 ==> 返回各任務的input、label
"""
def get_input_and_label(df_patient, window_size, stay_id):
    
    #宣告輸出格式
    # input_features = { 
    #     'dod': [],'dod_3day': [],'dod_7day':[], 'dod_30day': [], 'dod_60day': [], 'dod_90day': [],'DNR':[], 'SBT':[], 'Weaning_successful':[]   
    # }
    # labels = {
    #     'dod': [],'dod_3day': [],'dod_7day':[], 'dod_30day': [], 'dod_60day': [], 'dod_90day': [],'DNR':[], 'SBT':[], 'Weaning_successful':[]
    # }

    #宣告輸出格式
    input_features = { 
        'dod': [],'dod_3day': [],'dod_7day':[], 'dod_30day': [], 'dod_60day': [], 'dod_90day': [],'DNR':[], 'SBT':[], 'Weaning_successful':[], 
        'dialysis': [],'InvasiveVent': [],'Vasopressor':[],'DNR':[]  
    }
    labels = {
        'dod': [],'dod_3day': [],'dod_7day':[], 'dod_30day': [], 'dod_60day': [], 'dod_90day': [],'DNR':[], 'SBT':[], 'Weaning_successful':[], 
        'dialysis': [],'InvasiveVent': [],'Vasopressor':[],'DNR':[]
    }


    
#     input_features = { 
#         'dialysis': [],'InvasiveVent': [],'Vasopressor':[],'DNR':[]
#     }
#     labels = {
#         'dialysis': [],'InvasiveVent': [],'Vasopressor':[],'DNR':[]
#     }
    
    if len(df_patient) <= window_size:
        return input_features, labels
    
    #不當輸入的欄位
    excluded_columns = list(input_features.keys())
    excluded_columns = excluded_columns + ['InvasiveVent', 'tracheostomy', 'NonInvasiveVent', 'SupplementalOxygen', 'HFNC']
    excluded_columns = excluded_columns + ['date','use_vent','Weaning','sofa','Reintubation','stay_id','dnr','first_day_date']
    excluded_columns = excluded_columns + ['first_day_stay_id', 'first_day_use_vent', 'first_day_dod', 'first_day_dod_3day',
                                           'first_day_dod_7day', 'first_day_dod_30day', 'first_day_dod_60day', 'first_day_dod_90day', 
                                           'first_day_InvasiveVent', 'first_day_tracheostomy', 'first_day_NonInvasiveVent', 'first_day_SupplementalOxygen','first_day_SBT']      
    
    for i in range(0, len(df_patient)):
        if i + window_size >= len(df_patient):
            break
        
        #判斷該時窗是否皆使用呼吸器
        input_using_vent = df_patient['use_vent'][i:i + window_size].min() == 1
        
        df_input = df_patient.iloc[i:i + window_size, ~df_patient.columns.isin(excluded_columns)]
        #df_input = df_input.astype(float)
        input_X = df_input.values

        global df_columns
        if len(df_columns.columns) == 0:
            df_columns = df_input.copy()

        # labels_y = {
        #     'dod': df_patient.iloc[i + window_size:i + window_size + 1]['dod'].values,
        #     'dod_3day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_3day'].values,
        #     'dod_7day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_7day'].values,
        #     'dod_30day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_30day'].values,
        #     'dod_60day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_60day'].values,
        #     'dod_90day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_90day'].values,
        #     #'dialysis': df_patient.iloc[i + window_size:i + window_size + 1]['dialysis'].values,
        #     'DNR': df_patient.iloc[i + window_size:i + window_size + 1]['DNR'].values,
        #     'SBT': df_patient.iloc[i + window_size:i + window_size + 1]['SBT'].values,
        #     'Weaning_successful': df_patient.iloc[i + window_size:i + window_size + 1]['Weaning_successful'].values,
        # }

        labels_y = {
            'dod': df_patient.iloc[i + window_size:i + window_size + 1]['dod'].values,
            'dod_3day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_3day'].values,
            'dod_7day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_7day'].values,
            'dod_30day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_30day'].values,
            'dod_60day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_60day'].values,
            'dod_90day': df_patient.iloc[i + window_size:i + window_size + 1]['dod_90day'].values,
            #'dialysis': df_patient.iloc[i + window_size:i + window_size + 1]['dialysis'].values,
            'DNR': df_patient.iloc[i + window_size:i + window_size + 1]['DNR'].values,
            'SBT': df_patient.iloc[i + window_size:i + window_size + 1]['SBT'].values,
            'Weaning_successful': df_patient.iloc[i + window_size:i + window_size + 1]['Weaning_successful'].values,
            'dialysis': df_patient.iloc[i + window_size:i + window_size + 1]['dialysis'].values,
            'InvasiveVent': df_patient.iloc[i + window_size:i + window_size + 1]['InvasiveVent'].values,
            'Vasopressor' : df_patient.iloc[i + window_size:i + window_size + 1]['Vasopressor' ].values,
            'DNR': df_patient.iloc[i + window_size:i + window_size + 1]['DNR'].values,
        }

   
#         labels_y = {
#             'dialysis': df_patient.iloc[i + window_size:i + window_size + 1]['dialysis'].values,
#             'InvasiveVent': df_patient.iloc[i + window_size:i + window_size + 1]['InvasiveVent'].values,
#             'Vasopressor' : df_patient.iloc[i + window_size:i + window_size + 1]['Vasopressor' ].values,
#             'DNR': df_patient.iloc[i + window_size:i + window_size + 1]['DNR'].values,
#         }
        
        # 輸入資料沒有使用呼吸器的處理 => 將標籤設置為-1
        if input_using_vent == False:
            for key in ['Weaning_successful', 'SBT']:
                labels_y[key] = -1
        
        # 只保留標籤為1、0的樣本
        for key in input_features.keys():
            #assert labels_y[key] == 1 or labels_y[key] == 0
            if labels_y[key] == 1 or labels_y[key] == 0:
                input_features[key].append(input_X)
                labels[key].append(labels_y[key])
    
    #remove sample
    for key in input_features.keys():
        x_list = input_features[key]
        y_list = labels[key]
        x_list_select, y_list_select = remove_sample(x_list,y_list,key)
        input_features[key] = x_list_select
        labels[key] = y_list_select
        
    return input_features, labels

In [9]:
#task_name_list = ['dod','dod_3day','dod_7day','dod_30day','dod_60day','dod_90day','DNR','Weaning_successful','SBT']
#task_name_list = ['dialysis','InvasiveVent','Vasopressor','DNR']
task_name_list = ['dod','dod_3day','dod_7day','dod_30day','dod_60day','dod_90day','DNR','Weaning_successful','SBT','dialysis','InvasiveVent','Vasopressor','DNR']


input_features_list = []
labels_list = []
window_size = 3
distinct_stay_id = df['stay_id'].unique()

In [10]:
for stay_ids in tqdm(distinct_stay_id): 
    df_P = df[df['stay_id'] == stay_ids]

    df_P = df_P.reset_index(drop=True)
    df_P = correction_DNR(df_P)
    df_P = add_fisrt_day_columns(df_P, extend_col_num = 103)
    assert df_P.isna().any().any() == False, 'df_P中有缺失值'
    
    input_features, labels = get_input_and_label(df_P, window_size, stay_ids)
    input_features_list.append(input_features)
    labels_list.append(labels)


100%|██████████| 724/724 [00:06<00:00, 107.82it/s]


In [11]:
task_samples = {}
task_labels = {}

for i in tqdm(range(len(input_features_list))): 
    x_dict = input_features_list[i]
    y_dict = labels_list[i]
    
    for task, data in x_dict.items():
        if task not in task_samples:
            task_samples[task] = []
        task_samples[task] = task_samples[task] + data
        
    for task, label in y_dict.items():
        if task not in task_labels:
            task_labels[task] = []
        task_labels[task] = task_labels[task] + label        

100%|██████████| 724/724 [00:00<00:00, 94014.74it/s]


In [12]:
for task_name in task_name_list:
    task_samples[task_name] = np.array(task_samples[task_name])
    task_labels[task_name] = np.squeeze(task_labels[task_name])
    #print(task_samples[task_name].shape)
    
# print(len(task_samples[task_name]))
# print(task_samples[task_name])
# print(task_name)
# print(task_name_list)

In [13]:
def check_label_distribution (data_Y):
    count_1 = np.count_nonzero(data_Y == 1)
    count_0 = np.count_nonzero(data_Y == 0)
    count_others = np.count_nonzero((data_Y != 1) & (data_Y != 0))
    #print(f'count1:{count_1}....{count_0}')
    ratio_1 = round(count_1/len(data_Y)*100,2)
    ratio_0 = round(count_0/len(data_Y)*100,2)
    ratio_others = round(count_others/len(data_Y)*100,2)
    print(f'1=>{count_1}({ratio_1}%),  0=>{count_0}({ratio_0}%),  others=>{count_others}({ratio_others}%)')
    
    


    

In [ ]:
from datetime import datetime
date = datetime.today().strftime('%y%m%d')
#route = './data/sample' 

route = 'C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/sample' 
#route = 'C:/Users/USER/M1326168/MIMIC/DNR/20250312/data/sample' 


    
is_save = input('save? (y/n)')
if is_save == 'y':
    for task_name in task_name_list:
        print(f'---------------\n-----{task_name}-------')
        for data_type in ['train','validation','test']:
            len_of_sample = len(task_samples[task_name])
            feature_count = int(len(df_columns.columns)/2)
            if data_type == 'train':
                x = task_samples[task_name][:int(len_of_sample*0.8),:,:]
                y = task_labels[task_name][:int(len_of_sample*0.8)]
            elif data_type == 'validation':
                x = task_samples[task_name][int(len_of_sample*0.8):int(len_of_sample*0.9),:,:]
                y = task_labels[task_name][int(len_of_sample*0.8):int(len_of_sample*0.9)]
            else:
                x = task_samples[task_name][int(len_of_sample*0.9):,:,:]
                y = task_labels[task_name][int(len_of_sample*0.9):]
            assert x.shape[0] == y.shape[0],f'輸入、輸出樣本數不一致 => {x.shape[0]}!={y.shape[0]}'
            assert np.any(np.isnan(x)) == False
            assert np.any(np.isnan(y)) == False
            #print(f'{task_name}...{data_type}...{x.shape}')
            check_label_distribution(y)
            np.save(f'{route}/{data_type}_X_{task_name}.npy', x)
            np.save(f'{route}/{data_type}_Y_{task_name}.npy', y)
        
    print('Saved successfully....', date)
    
print("Finish....", datetime.now())

---------------
-----dod-------


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [ ]:
for i in df_columns.columns:
    print(i)

In [ ]:
check_label_distribution(task_labels['DNR'][:])
#check_label_distribution(task_labels['Full code'][:])

In [ ]:
#df_columns.iloc[:,:].to_csv(f"./data/sample/full_feature_name.csv", index=False)
df_columns.iloc[:,:].to_csv(f"C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/sample/full_feature_name.csv", index=False)
#df_columns.iloc[:,:].to_csv(f"C:/Users/USER/M1326168/MIMIC/DNR/20250312/data/sample/full_feature_name.csv", index=False)


# 用來處理特徵所屬category

In [ ]:
#df_columns = pd.read_csv("./data/sample/full_feature_name.csv")

df_columns = pd.read_csv("C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/sample/full_feature_name.csv")

#df_columns = pd.read_csv("C:/Users/USER/M1326168/MIMIC/DNR/20250312/data/sample/full_feature_name.csv")
print(len(df_columns.columns))

In [ ]:
df_col_with_category = pd.DataFrame({'feature_name': df_columns.columns})
df_col_with_category['category'] = "Can't find"

In [ ]:
# 定義函數來根據條件設置新欄位的值
def set_type(row):
    respiratory = [
        'FiO2', 'Minute Ventilation', 'Mean Airway Pressure', 'Peak Airway Pressure', 'PEEP', 'Plateau', 'use_vent_acc','use_vent_consecutive_7days',           
        'Tidal Volume', 'Respiratory Rate','Consecutive_avg_FiO2_Over50', 'Consecutive_avg_FiO2_Over60', 'use_vent_over3','first_day_use_vent_over3 '
    ] 
    
    GCS = [
        'Maintain GCS >= 8', 'RASS'
    ]
    target = [
        'Anxiety', 'Assess for anxiety, depression, or delirium which may contribut', 'Asystole', 'Blood Transfusion','dialysis_over3', 'dialysis_over7', 
        'Consult to pastoral care, social services, palliative care, psy', 'Early mobilization', 'Encourage family to participate in care', 
        'Encourage patient / family members to accept individual reponse', 'Encourage patient / family to set and verbalize care goals', 
        'Enlist family support / family sitters', 'Explanation', 'Family at bedside', 'Family Called', 'Family Conference', 'Family Member', 
        'Family Talked to MD', 'Family Talked to RN', 'Family Visited', 'Full code', 'Full resistance', 'Fully awake', 
        'Listen attentively when patient attempts to communicate', 'Liver Failure', "Manage environment for patient''s maximum comfort", 
        'Maintain GCS >= 8', 'No Proxy', 'No, not sedated', 'Palliative Care', 'Patient / family involved in treatment plan', 
        'Patient / family will have adequate support once CMO decision h', 'Patient / family will have support throughout the dying process', 
        'Patient / family will participate in planning and initiating cu', 'Patient refused', 'Patient Verbalized', 'Promote early mobility', 
        'Rehab', 'Rehabilitation', 'Social Service Involved', 'Social Services', 'Social Work', 'Spiritual', 'Thermoregulation', 'TPN', 'Tracheostomy', 
        'Tracheostomy tube', 'With family', 'With spouse', 'Withdraws','Spouse','Strength L Arm', 'Strength L Leg', 'Strength R Arm', 'Strength R Leg', 
        'Response', 'Coping/Knowledge Deficit  NCP - Expected outcomes',  'HFNC','Full_code_interval','Strength Arm','Strength Leg',
        'Muscle', 'Vocalization', 'Body Movements', 'Facial Expressions', 'CPOT(SUM)','ICU_Class_1','ICU_Class_2','ICU_Class_3','CVICU'
        ,
    ]
    #'DNR_after_in_ICU_day', 'early_DNR'目前不使用
    sofa = [
        'coagulation', 'liver', 'cardiovascular', 'cns', 'renal', 'respiration',
        'SOFA-coagulation', 'SOFA-liver', 'SOFA-cardiovascular', 'SOFA-cns', 'SOFA-renal', 'SOFA-respiration',
    ]
    patient = [
        'gender', 'age', 'race', 'apsiii'
    ]
    intakeoutput = [
        'total', 'Fluid_intake_value', 'Nutrition_Enteral_value', 'Urine_value'
    ]
    infusiondrug = [
        'Vasopressor', 'Relaxant', 'Sedation', 'PPI', 'Pain control', 'antibiotic','Consecutive_Vasopressor_Over3', 'Consecutive_Vasopressor_Over7'
    ]
    microlab = [
        'Aspergillus', 'Candida', 'Abdomen', 'Blood', 'Respiratory tract', 'Skin and soft tissue', 'Urinary tract', 'Others'
    ]
    diagnosis = [
        'MI', 'CHF', 'PVD', 'CVD', 'Dementia', 'CPD', 'RD', 'PUD', 'MLD', 'DM_acute', 'DM_Chronic', 'Hemiplegia', 'Renal', 
        'Malignancy', 'LD', 'MST', 'AIDS','dialysis','dialysis_since_last_event','dialysis_acc'
    ]
    vital_sign = [
        'SaO2', 'Heart Rate', 'Systemic Systolic', 'Systemic Diastolic', 'Systemic Mean'
    ]
    lab = [
        'Platelets x1000', 'WBC x1000', 'Hgb', 'PO2', 'PaCO2', 'Glucose', 'BUN', 'pH', 'Sodium', 'Potassium', 'Magnesium', 
        'Calcium', 'Chloride', 'creatinine', 'HCO3', 'Phosphate', 'PT-INR', 'Troponin - T', 'Ferritin', 'Transferrin', 'CRP', 
        'Fibrinogen', 'HDL', 'LDL', 'Total Cholesterol', 'Direct Bilirubin', 'Lactate', 'LDH', 'Lipase', 'Amylase', 'CPK', 'CPK-MB', 
        'BNP', 'PTT', 'Fe', 'TIBC', 'Fe/TIBC Ratio', 'Serum_Osmolality', 'TSH', 'T3', 'T4', 'Free T4', 'Ionized Calcium', 'Triglycerides', 
        'Cortisol', 'Uric Acid', 'Ammonia', 'Vitamin B12'
    ]
    
    cate_dict = {
        "respiratory":respiratory,
        "target":target, 
        "sofa":sofa, 
        "patient":patient, 
        "intakeoutput":intakeoutput, 
        "infusiondrug":infusiondrug, 
        "microlab":microlab, 
        "diagnosis":diagnosis, 
        "vital_sign":vital_sign, 
        "lab":lab,
        "GCS": GCS
    }
    
    for key, current_list in cate_dict.items():
        if row['feature_name'] in current_list:
            return key
        
        first_day_current_list = [f"first_day_{name}" for name in current_list]
        if row['feature_name'] in first_day_current_list:
            return key
    return "Can't find"

In [ ]:
# 幫每個特徵設置所屬category
df_col_with_category['category'] = df_col_with_category.apply(set_type, axis=1)
print(df_col_with_category)

In [ ]:
#檢測是否有找不到的category => 不能有
filtered_df = df_col_with_category[df_col_with_category['category'] == "Can't find"]
print(filtered_df)
print(filtered_df['feature_name'].tolist())

In [ ]:
#時序:114  靜態:256-114=142
if len(filtered_df) != 0:
    #df_col_with_category.to_csv(f"./data/sample/full_feature_name_with_category.csv", index=False)
    
    df_col_with_category.to_csv(f"C:/Users/USER/M1326168/MIMIC/DNR/20241002/data/sample/full_feature_name_with_category.csv", index=False)
    
    #df_col_with_category.to_csv(f"C:/Users/USER/M1326168/MIMIC/DNR/20250312/data/sample/full_feature_name_with_category.csv", index=False)
    print("儲存成功")